In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [5]:
from tra.models import *
from tra.test_runs_processing import *
from rep_portal.api import *
from functools import reduce
from django.contrib.auth.models import User

In [6]:
from tra.storage import get_storage_instance
from tra import utils

In [7]:
queryset = TestInstance.objects.all()
user = User.objects.get(username="plinta")
tsfilters = TestSetFilter.objects.filter(subscribers=user)
queryset = queryset.filter(
    reduce(lambda q, tsfilter: q | Q(test_set=tsfilter), tsfilters, Q())
)
queryset.count()

220

In [10]:
TestInstance.objects.filter(test_set__subscribers=user).count()

220

In [9]:
from itertools import chain 

auth_params = utils.get_rp_api_auth_params()
testset_filters = TestSetFilter.objects.exclude(subscribers=None)
branches = set([tsf.branch for tsf in testset_filters])
# ti_eligible_to_sync_all = list(chain(*[testset_filter.test_instances.all() for testset_filter in testset_filters]))
ti_eligible_to_sync_all = TestInstance.objects.none()
for testset_filter in testset_filters:
    ti_eligible_to_sync_all = ti_eligible_to_sync_all | testset_filter.test_instances.all() 
organizations = set([ti.organization for ti in ti_eligible_to_sync_all])

for organization in organizations:
    for branch in branches:
        print(TestInstance.objects.exclude(test_set__subscribers=None).filter(organization=organization, test_set__branch=branch).count())
        ti_eligible_to_sync = ti_eligible_to_sync_all.filter(organization=organization, test_set__branch=branch)
        print(ti_eligible_to_sync.count())
#         ti_noruns =  RepPortal(**auth_params).get_test_instances_for_present_feature_build_with_specified_status(
#             organization.name, status="no_run", release=branch.name, regression_test_types="rg_CIT,rg_CRT")
#         print(organization, branch)
#         ti_eligible_ids = set([ti.rp_id for ti in ti_eligible_to_sync])
#         ti_noruns_ids = set([ti["id"] for ti in ti_noruns])
#         ti_intersection = ti_eligible_ids.intersection(ti_noruns_ids)
#         print(ti_intersection)
#         TestInstance.objects.filter(rp_id__in=list(ti_intersection)).update(no_run_in_rp=True)
#         print(TestInstance.objects.filter(rp_id__in=ti_intersection))


110
110
103
103
110
110


NameError: name 'organization' is not defined

In [10]:
TestSetFilter.objects.exclude(subscribers=None)

<QuerySet [<TestSetFilter: Trunk - CLOUD_5G_I_LO_AP_LO_SANSA_FS_ECPRI_CMWV_TDD - 5GC001085-B_Intra-frequency_inter-gNB_neighbor_NRREL_addition_-_Previously_established_Xn>, <TestSetFilter: Trunk - CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD - CB008533_B_Multiple_UEs_Call_Setup_4CC_DL_CA>, <TestSetFilter: Trunk - CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD - 5GC002640_AD_DRX_MG_support_IODT>, <TestSetFilter: Trunk - CLOUD_5G_I_LO_AP_LO_SANSA_FS_ECPRI_CMWV_TDD - 5GC001085-A_Intra-frequency_intra-gNB_neighbor_NRREL_addition>, <TestSetFilter: Trunk - CLOUD_5G_O_E_AP_LO_SANSA_FS_ECPRI_CMWV_TDD - 5GC001085-A_Intra-frequency_intra-gNB_neighbor_NRREL_addition>, <TestSetFilter: Trunk - CLOUD_5G_O_E_AP_LO_SANSA_FS_ECPRI_CMWV_TDD - 5GC001085-B_Intra-frequency_inter-gNB_neighbor_NRREL_addition_-_Previously_established_Xn>, <TestSetFilter: Trunk - CLOUD_5G_I_LO_AP_LO_SANSA_FS_ECPRI_CMWV_TDD - 5GC001085-C_Intra-frequency_NRREL_deletion_(Pruning)_based_on_Measurement_Reports>, <TestSetFilter: